In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
!ls

Untitled.ipynb	df.csv


In [5]:
df = pd.read_csv('df.csv', sep=';')

# 1.

In [6]:
df.head()

,sender_id,platform_id,time_stamp,gender,reg_date
0,3207526951,6,16.03.2017 13:35,m,26.01.2017
1,3207526951,6,16.03.2017 9:09,m,26.01.2017
2,3207526951,6,16.03.2017 9:09,m,26.01.2017
3,3207526951,6,16.03.2017 12:13,m,26.01.2017
4,3207526951,6,15.03.2017 14:01,m,26.01.2017


In [7]:
df.shape

(768439, 5)

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768439 entries, 0 to 768438
Data columns (total 5 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   sender_id    768439 non-null  int64 
 1   platform_id  768439 non-null  int64 
 2   time_stamp   768439 non-null  object
 3   gender       768439 non-null  object
 4   reg_date     768439 non-null  object
dtypes: int64(2), object(3)
memory usage: 29.3+ MB


In [10]:
df.duplicated().sum()

518178

In [11]:
df.isnull().sum()

sender_id      0
platform_id    0
time_stamp     0
gender         0
reg_date       0
dtype: int64

In [12]:
df.nunique()

sender_id      26321
platform_id        2
time_stamp     20100
gender             3
reg_date          85
dtype: int64

In [13]:
df['gender'].unique()

array(['m', 'f', ' '], dtype=object)

In [14]:
df.describe()

,sender_id,platform_id
count,7.684390e+05,768439.000000
mean,3.212627e+09,6.419483
std,5.060134e+06,0.493475
min,3.201944e+09,6.000000
25%,3.208266e+09,6.000000
50%,3.214114e+09,6.000000
75%,3.216958e+09,7.000000
max,3.219311e+09,7.000000


In [15]:
df.describe(include='object')

,time_stamp,gender,reg_date
count,768439,768439,768439
unique,20100,3,85
top,14.03.2017 14:26,m,13.03.2017
freq,239,659569,31548
